## Analysis for the manuscript: data analysis

This analysis is conducted based on the Dauost experiment replication data, fielded in 2020. <br>
1) checking for randomization of treatment conditions <br>
2) experimental data analysis <br>
3) three-way interactions

In [1]:
import numpy as np
import pandas as pd
import utility as util

In [2]:
df_rep = pd.read_csv('../output/df_rep.csv')

In [3]:
df_rep.shape

(4545, 9)

In [4]:
df_rep.columns

Index(['id', 'condition', 'visit', 'over', 'outdoors', 'sex', 'marital',
       'age_group', 'education'],
      dtype='object')

### 1) checking for randomization of treatment conditions

In [5]:
democols = ['sex', 'marital', 'age_group', 'education']

for col in democols:
    print(df_rep[col].value_counts(dropna=False).sort_index())
    print("................")

1.0    2466
2.0    2079
Name: sex, dtype: int64
................
1.0    2995
2.0     382
3.0     658
4.0      44
5.0     466
Name: marital, dtype: int64
................
2.0      87
3.0     192
4.0     325
5.0     673
6.0    1326
7.0    1942
Name: age_group, dtype: int64
................
1.0      53
2.0     756
3.0    1722
4.0    1234
5.0     780
Name: education, dtype: int64
................


In [6]:
# recode demographic variables
df_rep['sex_r'] = df_rep['sex']

df_rep['marital_r'] = np.nan
df_rep.loc[df_rep['marital']==1, ['marital_r']] = 1 # married
df_rep.loc[np.isin(df_rep['marital'], [2, 3, 4, 5]), ['marital_r']] = 2 # not married

df_rep['age_group_r'] = np.nan
df_rep.loc[np.isin(df_rep['age_group'], [2, 3, 4, 5]), ['age_group_r']] = 1 # 18-54
df_rep.loc[df_rep['age_group']==6, ['age_group_r']] = 2 # 55-64
df_rep.loc[df_rep['age_group']==7, ['age_group_r']] = 3 # 65+

df_rep['education_r'] = np.nan
df_rep.loc[np.isin(df_rep['education'], [1, 2, 3]), ['education_r']] = 1 # less than college
df_rep.loc[np.isin(df_rep['education'], [4, 5]), ['education_r']] = 2 # college or more

In [7]:
# checking recoded variables
democols_r = ['sex_r', 'marital_r', 'age_group_r', 'education_r']

for col in democols_r:
    print(df_rep[col].value_counts(dropna=False).sort_index())
    print("................")

1.0    2466
2.0    2079
Name: sex_r, dtype: int64
................
1.0    2995
2.0    1550
Name: marital_r, dtype: int64
................
1.0    1277
2.0    1326
3.0    1942
Name: age_group_r, dtype: int64
................
1.0    2531
2.0    2014
Name: education_r, dtype: int64
................


In [9]:
for col in democols_r:
    util.crosstab_chisq(col, 'condition', df_rep, chisqtest=True)

condition,A,B
sex_r,,
1.0,54.1,54.4
2.0,45.9,45.6
Total n,"2,269","2,276"


*Chi-squared statistic = 0.0, degree of freedom = 1, p = 0.877*

-----

condition,A,B
marital_r,,
1.0,65.1,66.7
2.0,34.9,33.3
Total n,"2,269","2,276"


*Chi-squared statistic = 1.1, degree of freedom = 1, p = 0.296*

...

*Chi-squared statistic = 0.6, degree of freedom = 2, p = 0.751*

-----

condition,A,B
education_r,,
1.0,56.1,55.3
2.0,43.9,44.7
Total n,"2,269","2,276"


*Chi-squared statistic = 0.3, degree of freedom = 1, p = 0.593*

-----

### 2) experimental data analysis

In [10]:
# numeric -> character display
SD_cols = ['visit', 'over', 'outdoors']

for col in SD_cols:
    df_rep[col] = df_rep[[col]].replace([1, 2, 8, 11], ["1a. Yes", "2. No", "8. Unsure", "1b. Only when necessary/occasionally"])

In [11]:
# raw distributions for cross-tabs
for col in SD_cols:
    util.crosstab_chisq(col, 'condition', df_rep, chisqtest=False)

condition,A,B
visit,,
1a. Yes,61.1,62.0
1b. Only when necessary/occasionally,0.0,9.5
2. No,38.6,28.3
8. Unsure,0.3,0.2
Total n,"2,269","2,276"


-----

condition,A,B
over,,
1a. Yes,63.9,67.0
1b. Only when necessary/occasionally,0.0,10.6
2. No,35.6,21.7
8. Unsure,0.5,0.6
Total n,"2,269","2,276"


-----

condition,A,B
outdoors,,
1a. Yes,67.7,68.1
1b. Only when necessary/occasionally,0.0,9.3
2. No,31.9,22.1
8. Unsure,0.5,0.4
Total n,"2,269","2,276"


-----

In [12]:
# combining 1a, 1b, and 8 together as 1/yes
combine_three = {"1a. Yes": "1. Yes", 
                 "2. No": "2. No",
                 "1b. Only when necessary/occasionally": "1. Yes", 
                 "8. Unsure": "1. Yes"}

for col in SD_cols:
    df_rep[f'{col}_r'] = df_rep[col].map(combine_three)
    
recoded_cols = ['visit_r', 'over_r', 'outdoors_r'] # use recoded_cols for further analyses

In [13]:
# cross-tabs based on recodes
for col in recoded_cols:
    util.crosstab_chisq(col, 'condition', df_rep, chisqtest=True)

condition,A,B
visit_r,,
1. Yes,61.4,71.7
2. No,38.6,28.3
Total n,"2,269","2,276"


*Chi-squared statistic = 53.3, degree of freedom = 1, p = 0.0*

-----

condition,A,B
over_r,,
1. Yes,64.4,78.3
2. No,35.6,21.7
Total n,"2,269","2,276"


*Chi-squared statistic = 105.5, degree of freedom = 1, p = 0.0*

-----

condition,A,B
outdoors_r,,
1. Yes,68.1,77.9
2. No,31.9,22.1
Total n,"2,269","2,276"


*Chi-squared statistic = 54.0, degree of freedom = 1, p = 0.0*

-----

### 3) testing interactions

Testing whether the experimental effects vary by demographic variables ('sex', 'marital', 'age_group', 'education')

In [14]:
df_rep['condition_r'] = np.nan
df_rep.loc[df_rep['condition']=='A', ['condition_r']] = 1 # condition A
df_rep.loc[df_rep['condition']=='B', ['condition_r']] = 2 # condition B

In [15]:
# recode y as numerics
for col in recoded_cols:
    df_rep[col] = df_rep[[col]].replace(['1. Yes'], [1]) # yes have done the mentioned behavior
    df_rep[col] = df_rep[[col]].replace(['2. No'], [0]) # no haven not done the mentioned behavior

In [16]:
# to use StatsModels package in python, run the following
#X1 = df_rep[['condition_r', 'sex_r']].values
#X2 = df_rep[['condition_r', 'marital_r']].values
#X3 = df_rep[['condition_r', 'age_group_r']].values
#X4 = df_rep[['condition_r', 'education_r']].values

#y1 = df_rep['visit_r'].values
#y2 = df_rep['over_r'].values
#y3 = df_rep['outdoors_r'].values

In [17]:
df_rep['condition_r'].value_counts()

2.0    2276
1.0    2269
Name: condition_r, dtype: int64

#### - Running logistic regressions in R

DV: 'visit_r', 'over_r', 'outdoors_r' -> 1=have done; 2=haven't done <br>
IV: <br> 
1) "condition_r" -> 1=condition A, 2=condition B with excuse statement and guilty free response options <br>
2) demographic vars: 'sex_r', 'marital_r', 'age_group_r', 'education_r' -> see freq of recoded values above 

In [18]:
# use R for logistic regression tables
%load_ext rpy2.ipython

C:\ProgramData\Anaconda3\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  "The symbol '%s' is not in this R namespace/package." % name


In [22]:
%%R -i df_rep
dim(df_rep)

[1] 4545   17


In [23]:
%%R -i df_rep
print('VISIT - SEX')
summary(glm(visit_r ~ factor(condition_r) * factor(sex_r), family=binomial, data=df_rep))

[1] "VISIT - SEX"

Call:
glm(formula = visit_r ~ factor(condition_r) * factor(sex_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7098  -1.2883   0.7263   0.9197   1.0704  

Coefficients:
                                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.64708    0.06009  10.769  < 2e-16 ***
factor(condition_r)2                 0.55095    0.09026   6.104 1.03e-09 ***
factor(sex_r)2                      -0.39015    0.08670  -4.500 6.79e-06 ***
factor(condition_r)2:factor(sex_r)2 -0.16632    0.12774  -1.302    0.193    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5794.2  on 4544  degrees of freedom
Residual deviance: 5684.5  on 4541  degrees of freedom
AIC: 5692.5

Number of Fisher Scoring iterations: 4



In [24]:
%%R -i df_rep
print('VISIT - MARITAL')
summary(glm(visit_r ~ factor(condition_r) * factor(marital_r), family=binomial, data=df_rep))

[1] "VISIT - MARITAL"

Call:
glm(formula = visit_r ~ factor(condition_r) * factor(marital_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6206  -1.4191   0.7915   0.9536   1.0517  

Coefficients:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                              0.55218    0.05402  10.222  < 2e-16
factor(condition_r)2                     0.44777    0.07919   5.655 1.56e-08
factor(marital_r)2                      -0.24899    0.08996  -2.768  0.00564
factor(condition_r)2:factor(marital_r)2  0.03866    0.13259   0.292  0.77064
                                           
(Intercept)                             ***
factor(condition_r)2                    ***
factor(marital_r)2                      ** 
factor(condition_r)2:factor(marital_r)2    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Nul

In [25]:
%%R -i df_rep
print('VISIT - AGE')
summary(glm(visit_r ~ factor(condition_r) * factor(age_group_r), family=binomial, data=df_rep))

[1] "VISIT - AGE"

Call:
glm(formula = visit_r ~ factor(condition_r) * factor(age_group_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8540  -1.2035   0.7487   0.9644   1.1516  

Coefficients:
                                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                                1.12847    0.09271  12.173  < 2e-16
factor(condition_r)2                       0.39271    0.13820   2.842  0.00449
factor(age_group_r)2                      -0.61520    0.12285  -5.008 5.51e-07
factor(age_group_r)3                      -1.06735    0.11257  -9.482  < 2e-16
factor(condition_r)2:factor(age_group_r)2  0.15513    0.18282   0.849  0.39614
factor(condition_r)2:factor(age_group_r)3  0.07036    0.16624   0.423  0.67213
                                             
(Intercept)                               ***
factor(condition_r)2                      ** 
factor(age_group_r)2                      ***
fac

In [26]:
%%R -i df_rep
print('VISIT - EDUCATION')
summary(glm(visit_r ~ factor(condition_r) * factor(education_r), family=binomial, data=df_rep))

[1] "VISIT - EDUCATION"

Call:
glm(formula = visit_r ~ factor(condition_r) * factor(education_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6815  -1.3324   0.7466   0.9548   1.0300  

Coefficients:
                                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                                0.54914    0.05818   9.438  < 2e-16
factor(condition_r)2                       0.58584    0.08777   6.675 2.48e-11
factor(education_r)2                      -0.19198    0.08678  -2.212   0.0270
factor(condition_r)2:factor(education_r)2 -0.24838    0.12756  -1.947   0.0515
                                             
(Intercept)                               ***
factor(condition_r)2                      ***
factor(education_r)2                      *  
factor(condition_r)2:factor(education_r)2 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family

In [27]:
%%R -i df_rep
print('OVER - SEX')
summary(glm(over_r ~ factor(condition_r) * factor(sex_r), family=binomial, data=df_rep))

[1] "OVER - SEX"

Call:
glm(formula = over_r ~ factor(condition_r) * factor(sex_r), family = binomial, 
    data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8687  -1.3456   0.6192   0.8688   1.0180  

Coefficients:
                                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.77994    0.06147  12.689  < 2e-16 ***
factor(condition_r)2                 0.77430    0.09688   7.992 1.33e-15 ***
factor(sex_r)2                      -0.39286    0.08813  -4.458 8.28e-06 ***
factor(condition_r)2:factor(sex_r)2 -0.16059    0.13519  -1.188    0.235    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5444.5  on 4544  degrees of freedom
Residual deviance: 5288.3  on 4541  degrees of freedom
AIC: 5296.3

Number of Fisher Scoring iterations: 4



In [28]:
%%R -i df_rep
print('OVER - MARITAL')
summary(glm(over_r ~ factor(condition_r) * factor(marital_r), family=binomial, data=df_rep))

[1] "OVER - MARITAL"

Call:
glm(formula = over_r ~ factor(condition_r) * factor(marital_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7949  -1.3570   0.6675   0.8998   1.0078  

Coefficients:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                              0.69518    0.05520  12.594  < 2e-16
factor(condition_r)2                     0.69277    0.08468   8.181 2.82e-16
factor(marital_r)2                      -0.28233    0.09123  -3.095  0.00197
factor(condition_r)2:factor(marital_r)2 -0.02275    0.13935  -0.163  0.87033
                                           
(Intercept)                             ***
factor(condition_r)2                    ***
factor(marital_r)2                      ** 
factor(condition_r)2:factor(marital_r)2    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null 

In [29]:
%%R -i df_rep
print('OVER - AGE')
summary(glm(over_r ~ factor(condition_r) * factor(age_group_r), family=binomial, data=df_rep))

[1] "OVER - AGE"

Call:
glm(formula = over_r ~ factor(condition_r) * factor(age_group_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9535  -1.2815   0.6797   0.7966   1.0767  

Coefficients:
                                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                                 1.1896     0.0942  12.629  < 2e-16
factor(condition_r)2                        0.5578     0.1453   3.840 0.000123
factor(age_group_r)2                       -0.5575     0.1249  -4.465 8.02e-06
factor(age_group_r)3                       -0.9481     0.1140  -8.313  < 2e-16
factor(condition_r)2:factor(age_group_r)2   0.1576     0.1922   0.820 0.412310
factor(condition_r)2:factor(age_group_r)3   0.1858     0.1746   1.064 0.287286
                                             
(Intercept)                               ***
factor(condition_r)2                      ***
factor(age_group_r)2                      ***
facto

In [30]:
%%R -i df_rep
print('OVER - EDUCATION')
summary(glm(over_r ~ factor(condition_r) * factor(education_r), family=binomial, data=df_rep))

[1] "OVER - EDUCATION"

Call:
glm(formula = over_r ~ factor(condition_r) * factor(education_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8458  -1.3638   0.6339   0.8870   1.0018  

Coefficients:
                                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                                0.72990    0.05983  12.200  < 2e-16
factor(condition_r)2                       0.77272    0.09444   8.182 2.78e-16
factor(education_r)2                      -0.30180    0.08822  -3.421 0.000624
factor(condition_r)2:factor(education_r)2 -0.16159    0.13495  -1.197 0.231146
                                             
(Intercept)                               ***
factor(condition_r)2                      ***
factor(education_r)2                      ***
factor(condition_r)2:factor(education_r)2    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family t

In [31]:
%%R -i df_rep
print('OUTDOORS - SEX')
summary(glm(outdoors_r ~ factor(condition_r) * factor(sex_r), family=binomial, data=df_rep))

[1] "OUTDOORS - SEX"

Call:
glm(formula = outdoors_r ~ factor(condition_r) * factor(sex_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9016  -1.3723   0.7728   0.8276   0.9943  

Coefficients:
                                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          1.05564    0.06521  16.188  < 2e-16 ***
factor(condition_r)2                 0.57331    0.10072   5.692 1.25e-08 ***
factor(sex_r)2                      -0.60837    0.09105  -6.682 2.36e-11 ***
factor(condition_r)2:factor(sex_r)2 -0.12510    0.13734  -0.911    0.362    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5301.5  on 4544  degrees of freedom
Residual deviance: 5149.8  on 4541  degrees of freedom
AIC: 5157.8

Number of Fisher Scoring iterations: 4



In [32]:
%%R -i df_rep
print('OUTDOORS - MARITAL')
summary(glm(outdoors_r ~ factor(condition_r) * factor(marital_r), family=binomial, data=df_rep))

[1] "OUTDOORS - MARITAL"

Call:
glm(formula = outdoors_r ~ factor(condition_r) * factor(marital_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7949  -1.4437   0.6675   0.8453   0.9327  

Coefficients:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                              0.84537    0.05674  14.899  < 2e-16
factor(condition_r)2                     0.54258    0.08569   6.332 2.43e-10
factor(marital_r)2                      -0.23825    0.09358  -2.546   0.0109
factor(condition_r)2:factor(marital_r)2 -0.12864    0.14017  -0.918   0.3588
                                           
(Intercept)                             ***
factor(condition_r)2                    ***
factor(marital_r)2                      *  
factor(condition_r)2:factor(marital_r)2    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

 

In [33]:
%%R -i df_rep
print('OUTDOORS - AGE')
summary(glm(outdoors_r ~ factor(condition_r) * factor(age_group_r), family=binomial, data=df_rep))

[1] "OUTDOORS - AGE"

Call:
glm(formula = outdoors_r ~ factor(condition_r) * factor(age_group_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9918  -1.3328   0.6318   0.8446   1.0296  

Coefficients:
                                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                                 1.5099     0.1035  14.590  < 2e-16
factor(condition_r)2                        0.3258     0.1541   2.115   0.0345
factor(age_group_r)2                       -0.7192     0.1334  -5.391 7.01e-08
factor(age_group_r)3                       -1.1517     0.1221  -9.431  < 2e-16
factor(condition_r)2:factor(age_group_r)2   0.3346     0.2014   1.662   0.0966
factor(condition_r)2:factor(age_group_r)3   0.1633     0.1814   0.900   0.3679
                                             
(Intercept)                               ***
factor(condition_r)2                      *  
factor(age_group_r)2                      *

In [34]:
%%R -i df_rep
print('OUTDOORS - EDUCATION')
summary(glm(outdoors_r ~ factor(condition_r) * factor(education_r), family=binomial, data=df_rep))

[1] "OUTDOORS - EDUCATION"

Call:
glm(formula = outdoors_r ~ factor(condition_r) * factor(education_r), 
    family = binomial, data = df_rep)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7822  -1.4803   0.7455   0.8553   0.9022  

Coefficients:
                                          Estimate Std. Error z value Pr(>|z|)
(Intercept)                                0.81718    0.06080  13.441  < 2e-16
factor(condition_r)2                       0.54250    0.09266   5.854 4.79e-09
factor(education_r)2                      -0.12855    0.09060  -1.419    0.156
factor(condition_r)2:factor(education_r)2 -0.09283    0.13581  -0.684    0.494
                                             
(Intercept)                               ***
factor(condition_r)2                      ***
factor(education_r)2                         
factor(condition_r)2:factor(education_r)2    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial 